In [ ]:
from dotenv import find_dotenv, load_dotenv
from langfuse import get_client

_ = load_dotenv(find_dotenv())
langfuse = get_client()

In [ ]:
# get TraceWithDetails
traces = langfuse.api.trace.list(limit=1)
trace_data = traces.data[0]
trace_id = trace_data.id
print(f"trace_id: {trace_data.id}")
print(f"trace input: {trace_data.input}")
print(f"trace data type: {type(trace_data)}")

In [ ]:
trace_data

In [ ]:
TraceWithFullDetails = langfuse.api.trace.get(trace_id)
TraceWithFullDetails

In [ ]:
observations = TraceWithFullDetails.observations
len(observations)

In [ ]:
observations

In [ ]:
for obv in observations:
    print(obv.name)

In [ ]:
def retrieve_parser(obv):
    start_time = obv.start_time
    end_time = obv.end_time
    query_text = obv.input
    output_nodes = obv.output
    num_doc = len(output_nodes)
    
    meta_dict = obv.metadata['attributes']
    docs = []
    for idx in range(num_doc):
        rv = {
            'id': meta_dict[f'retrieval.documents.{idx}.document.id'],
            'content': meta_dict[f'retrieval.documents.{idx}.document.content'],
            'score': meta_dict[f'retrieval.documents.{idx}.document.score'],
        }
        docs.append(rv)
        
    rv = {
        'start_time': start_time,
        'end_time': end_time,
        'query': query_text,
        'output_nodes': output_nodes,
        'num_doc': num_doc,
        'docs': docs
    }
    return rv

def chat_parser(obv):
    start_time = obv.start_time
    end_time = obv.end_time
    model_name = obv.model
    num_message = len(obv.input['messages'])
    attributes = obv.metadata['attributes']
    messages = []
    for idx in range(num_message):
        rv = {
            'role': attributes[f'llm.input_messages.{idx}.message.role'],
            'content': attributes[f'llm.input_messages.{idx}.message.content'],
        }
        messages.append(rv)
    rv = {
        'start_time': start_time,
        'end_time': end_time,
        'model_name': model_name,
        'num_message': num_message,
        'messages': messages,
    }
    return rv

In [ ]:
data = {}
for obv in observations:
    name = obv.name
    if name == 'VectorIndexRetriever.retrieve':
        data['retrieve_parsed'] = retrieve_parser(obv)
    elif name == 'OpenAI.chat':
        data['chat_parsed'] = chat_parser(obv)

data